<a href="https://colab.research.google.com/github/acdc2019/algo-trading/blob/main/python/notebooks/strategies/rsi_strategy/RSIStrategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **RSI Strategy**
## **15 mins strategy**
* Step 1: A 15 min candle has to close above 70 RSI value.
* Step 2: Above High of this candle Buy signal is generated.
* Step 3: Stop Loss of the trade has to be below the earlier 15 min candles low.
* Step 4: Volume of the candle in which buy signal is generated has to be atleast 2 to 3 times of earlier 5 candles (***Q: average of earlier 5 candle volume?***)
* Step 5: Hourly candles RSI should be above 50 for confirmation.
* Step 6: In the last 5 candles the stock movement should not be more than 4% of the day movement
* Step 7: If Stop Loss of the candle is less than 6000 INR then the trade should be executed

#### **Strategy Parameters**
* window_start, window_end: Dates between which to look for signal
* rsi_15min = 70
* rsi_60min = 50
* stop_loss = 6000
* back_candles = 5
* volume_multiple = 2 or 3 times
* daily_movement_pct = 4

Install libs

In [3]:
!pip install ta
!pip install pandas
!pip install plotly

  Created wheel for ta: filename=ta-0.8.0-py3-none-any.whl size=28895 sha256=685ff9bda965a56f52a99723e19ea72185d26f3f52ffa4420cc84faf6a547050
  Stored in directory: /root/.cache/pip/wheels/7e/da/86/65cba22446ae2ef148de2079907264ef27feecfb7f51a45e0d
Successfully built ta
     |████████████████████████████████| 64 kB 2.4 MB/s 


Import Libs

In [4]:
import pandas as pd
from ta.momentum import RSIIndicator

In [5]:
def get_back_candles(df, index, back_count):
  loc = df.index.get_loc(index)
  return df.iloc[loc-1-back_count:loc-1]

## Load 15 min and 60 min stock data and calculate RSI

In [6]:
file_15min = '/content/stock_data/LT-HIST.csv'
file_60min = '/content/stock_data/LT-HIST-60M.csv'

df_15min = pd.read_csv(file_15min, parse_dates=['Date'], index_col=['Date'])
rsi = RSIIndicator(df_15min['Close']).rsi()
df_15min = df_15min.assign(rsi=rsi.values)
df_15min.tail()


,Open,High,Low,Close,Volume,rsi
Date,,,,,,
2021-12-17 14:15:00+05:30,1870.00,1870.80,1857.00,1857.40,68357,53.205167
2021-12-17 14:30:00+05:30,1857.45,1859.45,1846.15,1847.50,87319,46.057840
2021-12-17 14:45:00+05:30,1847.95,1848.35,1842.00,1847.25,159813,45.890192
2021-12-17 15:00:00+05:30,1847.20,1851.15,1841.70,1846.80,382071,45.568664
2021-12-17 15:15:00+05:30,1847.35,1852.00,1843.10,1846.00,355192,44.965492


In [7]:
df_60min = pd.read_csv(file_60min, parse_dates=['Date'], index_col=['Date'])
rsi = RSIIndicator(df_60min['Close']).rsi()
df_60min = df_60min.assign(rsi=rsi.values)
df_60min.tail()

,Open,High,Low,Close,Volume,rsi
Date,,,,,,
2021-12-17 11:15:00+05:30,1842.05,1846.7,1831.70,1839.25,260020,34.188135
2021-12-17 12:15:00+05:30,1839.80,1847.0,1836.90,1839.10,187805,34.127535
2021-12-17 13:15:00+05:30,1839.00,1874.4,1836.85,1870.50,327162,52.934661
2021-12-17 14:15:00+05:30,1870.00,1870.8,1841.70,1846.80,697560,42.963950
2021-12-17 15:15:00+05:30,1847.35,1852.0,1843.10,1846.00,355192,42.671769


## Set Strategy Parameters

In [8]:
window_start = '2021-12-01 00:00:00'
window_end = '2021-12-18 00:00:00'
rsi_15min = 70
rsi_60min = 50
stop_loss = 6000
back_candles = 5
volume_multiple = 2 # 2 or 3 times
daily_movement_pct = 4


## Actual Strategy Implementation

### Step 1: A 15 min candle has to close above 70 RSI value
Get all the candles in the window that close above required RSI value

In [15]:
curr_window_df = df_15min[window_start:window_end]

rsi_filter = (curr_window_df['rsi'] > rsi_15min)
df = curr_window_df[rsi_filter]

bullish_filter = (df['Close'] > df['Open'])
df = df[bullish_filter]

df.head()


,Open,High,Low,Close,Volume,rsi
Date,,,,,,
2021-12-03 09:15:00+05:30,1799.95,1837.10,1799.05,1835.00,500391,83.728661
2021-12-03 09:30:00+05:30,1834.70,1844.80,1829.00,1839.85,522158,84.787036
2021-12-08 09:45:00+05:30,1829.40,1835.45,1829.00,1834.50,231217,71.301778
2021-12-08 12:00:00+05:30,1831.60,1836.95,1831.60,1836.70,60756,70.054540
2021-12-09 11:15:00+05:30,1844.90,1856.00,1842.00,1855.45,191935,75.005132



### Step 4: Volume check

In [10]:
temp_df = pd.DataFrame(columns=['Open','High','Low','Close','Volume','rsi','MeanVolume'])
temp_df.index.name = 'Date'
back_candles_df = pd.DataFrame()
for index, row in df.iterrows():
  # Get mean volume of previous candles
  back_candles_df = get_back_candles(curr_window_df, index, back_candles)
  # print('Current candle:', index)
  # print('Back candles:', back_candles_df)
  mean_volume = back_candles_df['Volume'].mean()
  row['MeanVolume'] = mean_volume

  # Compare volume against previous mean volume
  if(row['Volume'] > volume_multiple*mean_volume):
    # print('Current Volume:', row['Volume'], 'greather than 2 times mean volume', mean_volume)
    temp_df.loc[index] = row

df = temp_df[['Open','High','Low','Close','Volume','rsi']]
temp_df

,Open,High,Low,Close,Volume,rsi,MeanVolume
Date,,,,,,,
2021-12-03 09:15:00+05:30,1799.95,1837.1,1799.05,1835.00,500391.0,83.728661,100603.6
2021-12-03 09:30:00+05:30,1834.70,1844.8,1829.00,1839.85,522158.0,84.787036,152847.4
2021-12-09 11:15:00+05:30,1844.90,1856.0,1842.00,1855.45,191935.0,75.005132,70122.0
2021-12-09 12:30:00+05:30,1857.80,1865.3,1857.20,1864.00,438863.0,77.463257,168215.6
2021-12-15 14:00:00+05:30,1886.00,1898.6,1884.10,1897.55,134844.0,76.872663,53974.2


### Step 5: Hourly candle RSI check

In [11]:
temp_df = pd.DataFrame(columns=df.columns)
hourly_df = pd.DataFrame(columns=df.columns)
temp_df.index.name = 'Date'
hourly_df.index.name = 'Date'
for index, row in df.iterrows():
  # Get the corresponding hourly candle for given 15min candle
  if(index in df_60min.index):
    # Candle already exists
    rsi = df_60min.loc[index]['rsi']
    hourly_df.loc[index] = df_60min.loc[index]
  else:
    # Update the index to the previous hourly candle index
    fromTime = index-pd.Timedelta(45,unit='min')
    temp_row = df_60min[df_60min.index >= fromTime].iloc[0]
    # print(temp_df)
    rsi = temp_row['rsi']
    hourly_df.loc[fromTime] = temp_row

  if(rsi > rsi_60min):
    # This 15min candle is eligible for signal
    # print('Hourly candle RSI is greater than 50..')
    temp_df.loc[index] = row

df = temp_df
df,hourly_df

(                              Open    High  ...    Volume        rsi
 Date                                        ...                     
 2021-12-03 09:15:00+05:30  1799.95  1837.1  ...  500391.0  83.728661
 2021-12-03 09:30:00+05:30  1834.70  1844.8  ...  522158.0  84.787036
 2021-12-09 11:15:00+05:30  1844.90  1856.0  ...  191935.0  75.005132
 2021-12-09 12:30:00+05:30  1857.80  1865.3  ...  438863.0  77.463257
 2021-12-15 14:00:00+05:30  1886.00  1898.6  ...  134844.0  76.872663
 
 [5 rows x 6 columns],
                               Open     High  ...     Volume        rsi
 Date                                         ...                      
 2021-12-03 09:15:00+05:30  1799.95  1844.80  ...  1573021.0  65.699785
 2021-12-03 08:45:00+05:30  1799.95  1844.80  ...  1573021.0  65.699785
 2021-12-09 11:15:00+05:30  1844.90  1859.70  ...   549496.0  70.080938
 2021-12-09 11:45:00+05:30  1857.95  1874.55  ...   747975.0  74.140649
 2021-12-15 13:15:00+05:30  1882.70  1898.60  ...   3

### Step 6: Stock movement check

In [12]:
temp_df = pd.DataFrame(columns=df.columns)
temp_df.index.name = 'Date'

for index, row in df.iterrows():
  prev_candles = get_back_candles(df_15min, index, 5)
  min_low = prev_candles['Low'].min()
  daily_movement = row['High'] - min_low
  tanaji_pct = (daily_movement/min_low)*100
  print('daily_movement:', daily_movement,'tanaji_pct:', tanaji_pct)
  if(tanaji_pct < daily_movement_pct):
    temp_df.loc[index] = row

temp_df

daily_movement: 53.549999999999955 tanaji_pct: 3.002438956014687
daily_movement: 60.299999999999955 tanaji_pct: 3.379097786494814
daily_movement: 27.450000000000045 tanaji_pct: 1.5011894670640697
daily_movement: 28.5 tanaji_pct: 1.5516114982578397
daily_movement: 20.59999999999991 tanaji_pct: 1.0969116080937118


,Open,High,Low,Close,Volume,rsi
Date,,,,,,
2021-12-03 09:15:00+05:30,1799.95,1837.1,1799.05,1835.00,500391.0,83.728661
2021-12-03 09:30:00+05:30,1834.70,1844.8,1829.00,1839.85,522158.0,84.787036
2021-12-09 11:15:00+05:30,1844.90,1856.0,1842.00,1855.45,191935.0,75.005132
2021-12-09 12:30:00+05:30,1857.80,1865.3,1857.20,1864.00,438863.0,77.463257
2021-12-15 14:00:00+05:30,1886.00,1898.6,1884.10,1897.55,134844.0,76.872663


## **Final Signal**

In [13]:
for index, row in df.iterrows():
  prev_candle = get_back_candles(df_15min, index, 1)
  stop_loss = prev_candle.iloc[0]['Low']
  print('For candle:', index, ', rsi:', row['rsi'], 'Entry Price', row['High'], 'Stop Loss:', stop_loss)

For candle: 2021-12-03 09:15:00+05:30 , rsi: 83.72866060951856 Entry Price 1837.1 Stop Loss: 1784.5
For candle: 2021-12-03 09:30:00+05:30 , rsi: 84.78703627467281 Entry Price 1844.8 Stop Loss: 1786.85
For candle: 2021-12-09 11:15:00+05:30 , rsi: 75.00513243277811 Entry Price 1856.0 Stop Loss: 1836.0
For candle: 2021-12-09 12:30:00+05:30 , rsi: 77.46325689515533 Entry Price 1865.3 Stop Loss: 1855.3
For candle: 2021-12-15 14:00:00+05:30 , rsi: 76.87266339019192 Entry Price 1898.6 Stop Loss: 1880.0


In [14]:
import plotly.graph_objects as go
import plotly.offline as py

# df.reset_index(inplace=True)
# print(df['Date'])
'''data=[go.Candlestick(x=df['Date'],
                     open=df['Open'],
                     high=df['High'],
                     low=df['Low'],
                     close=df['Close'])]

py.iplot(data)
'''

for index, row in df.iterrows():
  temp1 = get_back_candles(df_15min, index, 6)
  candles = pd.concat([temp1])
  candles.loc[index] = row

  candles['DateStr'] = candles.index.strftime('%Y-%m-%d %H:%M:%S')

  fig=go.Figure(data=[go.Candlestick(x=candles['DateStr'],
                     open=candles['Open'],
                     high=candles['High'],
                     low=candles['Low'],
                     close=candles['Close'])])
  fig.update_xaxes(type='category', rangeslider=dict(visible=False))

  py.iplot(fig)